In [1]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [28]:
import os
import sys
sys.path.append('../')

import pandas as pd
pd.set_option('display.max_columns', 200)

import src.generate_report as report_gen

In [17]:
report = report_gen.Report("04272022")
report.run()

KeyError: 'win_rate'

In [20]:
report.results["serves"]["top"]

{'Hagen': 10.1, 'True': 9.8, 'Devin': 9.7, 'Bruno': 9.5, 'Nate': 9.2}

In [11]:
flag = True

In [14]:
not flag

False

In [32]:
late, prev = report.compare_stats()
late.set_index("name",inplace=True)
prev.set_index("name",inplace=True)

In [33]:
late[late["name"] == player]

KeyError: 'name'

In [37]:
for player in late.index.unique():
    per_player = late[late.index == player] -  prev[prev.index == player]

In [38]:
per_player

,points_for,points_against,serves,aces,missed_serves,received,pass_rating,swings,swing kills,blocks,block kills,bump kills,hitting_errors,errors,serving_percentage,kills,hitting_efficiency,serve_receive_rating,positive,effectiveness,n,win_rate,ace2error
name,,,,,,,,,,,,,,,,,,,,,,,
Jason,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
late - prev

,points_for,points_against,serves,aces,missed_serves,received,pass_rating,swings,swing kills,blocks,block kills,bump kills,hitting_errors,errors,serving_percentage,kills,hitting_efficiency,serve_receive_rating,positive,effectiveness,n,win_rate,ace2error
name,,,,,,,,,,,,,,,,,,,,,,,
Brandon,-0.4,0.0,0.1,-0.1,-0.1,0.0,-0.4,-0.5,-0.2,0.0,0.0,-0.1,-0.2,-0.2,0.6,-0.2,-0.2,0.0,-0.3,-0.1,5.0,-0.2,0.0
Bruno,0.2,-0.3,0.0,0.1,0.0,-0.4,-0.6,-0.1,-0.1,-0.1,-0.1,-0.1,0.0,0.0,0.2,-0.2,-1.5,0.1,-0.1,-0.1,5.0,3.6,0.1
Carlos,-0.5,0.0,-0.2,0.0,0.0,0.1,-0.4,-0.3,-0.1,0.0,0.0,0.0,0.0,0.3,0.1,0.0,0.8,0.0,0.0,-0.3,3.0,-2.7,0.0
Cody,-0.2,-0.6,0.0,0.0,-0.1,-0.3,-0.7,-0.3,-0.1,-0.1,0.0,0.0,-0.1,-0.2,1.5,-0.1,0.8,-0.1,-0.2,0.0,10.0,3.4,0.0
Devin,0.2,0.2,0.1,0.1,0.0,0.1,0.1,-0.2,-0.1,0.0,-0.1,0.0,-0.1,0.0,0.0,-0.1,0.1,0.0,-0.1,-0.1,4.0,-0.7,0.0
German,0.0,0.0,-0.2,0.0,0.1,-0.2,0.0,-0.2,-0.1,0.1,0.0,0.1,0.3,0.3,-1.7,0.0,0.6,0.1,0.0,-0.4,14.0,-2.0,-0.1
Hagen,-0.3,-0.1,-0.1,0.0,-0.1,0.2,0.4,-0.1,-0.1,-0.1,-0.1,0.0,-0.1,-0.1,0.7,-0.2,-1.2,0.0,-0.2,-0.1,13.0,-1.5,-0.1
Jake,0.2,0.1,0.1,0.0,0.2,-0.2,-0.6,-0.3,-0.1,0.0,0.0,0.0,-0.1,0.1,-1.5,-0.1,-0.2,-0.1,-0.1,-0.3,2.0,2.8,-0.1
Jason,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
report.get_winningest_team(min_games=5)

{'Hagen/Tommy': ('100.0%', 9),
 'Brandon/Hagen': ('88.9%', 9),
 'Devin/Hagen': ('86.7%', 15),
 'Carlos/Hagen': ('83.3%', 12),
 'Bruno/Hagen': ('82.4%', 17)}